make synthethis data

In [ ]:
import pandas as pd
import random

# Elemen untuk laporan valid (dibuat lebih detail dan realistis)
bencana = [
    "Gempa bumi", "Banjir bandang", "Tanah longsor", "Tsunami", "Kekeringan","Banjir"
    "Angin puting beliung", "Kebakaran hutan", "Letusan gunung berapi", "Banjir rob", "Longsor lahar dingin"
]
magnitudo = [f"{random.uniform(3.0, 8.0):.1f} SR" for _ in range(20)]  # Magnitudo acak untuk gempa
lokasi = [
    "Yogyakarta", "Jakarta", "Bandung", "Garut", "Aceh", "Lombok", "Palu", "Bali",
    "Sumatra Barat", "Kalimantan Selatan", "Sulawesi Utara", "Papua", "Jawa Timur",
    "Banten", "Lampung", "NTT", "NTB", "Maluku", "Riau", "Bengkulu"
]
kecamatan = [
    "Kecamatan Cangkringan", "Kecamatan Cisarua", "Kecamatan Sukajaya", "Kecamatan Pamekasan",
    "Kecamatan Tanjung", "Kecamatan Ubud", "Kecamatan Soreang", "Kecamatan Ciputat",
    "Kecamatan Kalianda", "Kecamatan Woha", "Kecamatan Prambanan", "Kecamatan Karangasem",
    "Kecamatan Banyuwangi", "Kecamatan Serang", "Kecamatan Pesawaran"
]
desa = [
    "Desa Sukamaju", "Desa Mekarwangi", "Desa Cikahuripan", "Desa Pulosari", "Desa Jaya Baru",
    "Desa Tegalrejo", "Desa Sidomulyo", "Desa Gunungkidul", "Desa Pasirjaya", "Desa Margodadi"
]
dampak = [
    f"Sebanyak {random.randint(10, 200)} rumah mengalami kerusakan ringan hingga berat",
    f"Tercatat {random.randint(1, 50)} warga mengalami luka-luka",
    f"Sedikitnya {random.randint(1, 20)} korban jiwa dilaporkan",
    "Akses jalan utama terputus akibat material longsor",
    "Pasokan listrik terhenti selama beberapa jam di wilayah terdampak",
    f"Lebih dari {random.randint(50, 500)} hektar lahan pertanian terendam banjir",
    f"Sebanyak {random.randint(1, 10)} desa terisolasi akibat akses jalan yang terputus",
    f"Sebanyak {random.randint(20, 300)} warga terpaksa mengungsi ke tempat yang lebih aman",
    "Banyak pohon tumbang yang menutup akses jalan utama",
    f"Kerugian material diperkirakan mencapai Rp {random.randint(1, 100)} miliar",
    f"Sebanyak {random.randint(1, 50)} ternak dilaporkan mati akibat bencana",
    "Pasokan air bersih terganggu di beberapa wilayah",
    f"Sebanyak {random.randint(1, 20)} sekolah mengalami kerusakan",
    "Jembatan utama yang menghubungkan dua desa ambruk"
]
waktu = [
    "pada pagi hari ini", "pada malam hari tadi", "pada sore hari kemarin", "dua hari yang lalu",
    f"pada pukul {random.randint(0, 23):02d}.{random.randint(0, 59):02d} WIB",
    "saat subuh tadi", "pada malam hari kemarin", "pada siang hari tadi", "seminggu yang lalu",
    "tiga hari yang lalu"
]
penutup_valid = [
    "Pihak BPBD setempat langsung menangani kejadian ini dengan mendirikan posko darurat.",
    "Tim SAR dikerahkan ke lokasi untuk membantu proses evakuasi warga.",
    "Warga diminta tetap waspada terhadap potensi bencana susulan di wilayah tersebut.",
    "Pemerintah daerah segera menyalurkan bantuan logistik berupa makanan dan selimut.",
    "Hingga saat ini belum ada laporan korban jiwa, namun pendataan masih dilakukan.",
    "Proses evakuasi warga yang terdampak masih berlangsung hingga malam ini.",
    "Warga diimbau untuk tetap tenang dan mengikuti arahan dari petugas di lapangan.",
    "Bantuan darurat berupa tenda dan makanan mulai didistribusikan ke warga terdampak.",
    "Petugas masih melakukan pendataan kerugian dan korban di lokasi kejadian.",
    "Posko darurat telah didirikan untuk menampung warga yang kehilangan tempat tinggal."
]

# Elemen untuk laporan invalid (hoax, dibuat lebih terstruktur tapi tetap absurd)
bencana_hoax = [
    "Gempa bumi raksasa akibat ulah alien", "Banjir darah dari langit", "Tsunami api yang membakar laut",
    "Hujan meteor yang menghasilkan emas", "Angin topan membawa naga raksasa",
    "Kekeringan akibat kutukan dukun", "Lava meletus dari gunung es di tengah kota",
    "Hujan emas yang membuat warga kaya mendadak", "Banjir cokelat dari sungai",
    "Gempa bumi akibat ledakan di bulan", "Hujan berlian yang membuat warga takut",
    "Tsunami pelangi yang mengubah warna laut", "Angin topan ungu yang membuat orang terbang",
    "Kebakaran hutan akibat napas naga", "Letusan gunung berapi menghasilkan permen"
]
lokasi_hoax = [
    "di perairan Atlantis yang hilang", "di kota Bikini Bottom", "di Langit Ketujuh",
    "di dasar Laut Pasifik", "di permukaan Planet Mars", "di Kota Gaib Saranjana",
    "di Dunia Paralel", "di Kerajaan Atlantis yang tenggelam", "di Galaksi Andromeda",
    "di Benua Mu yang hilang", "di Kota Emas El Dorado", "di Lautan Awan",
    "di Dimensi Kelima", "di Kutub Selatan", "di pusat inti Bumi"
]
dampak_hoax = [
    "mengakibatkan semua ikan di laut mati mendadak", "menyebabkan alien muncul dari dalam air",
    "mengakibatkan dinosaurus hidup kembali di tengah kota", "menyebabkan semute berubah menjadi raksasa",
    "mengakibatkan pohon-pohon berubah menjadi emas", "menyebabkan langit berubah warna menjadi ungu",
    "mengakibatkan semua orang di wilayah itu bisa terbang", "menyebabkan ikan-ikan mulai berbicara",
    "mengakibatkan gunung berubah menjadi cokelat leleh", "menyebabkan warga panik dan ketakutan",
    "mengakibatkan burung-burung menjadi raksasa", "menyebabkan air laut berubah menjadi sirup manis",
    "mengakibatkan tanah berubah menjadi permen karet", "menyebabkan hewan-hewan mulai menari di jalan",
    "mengakibatkan semua warga menjadi takut dan bersembunyi"
]
waktu_hoax = [
    "pada malam hari tadi", "pada pagi hari ini", "pada sore hari kemarin", "dua hari yang lalu",
    f"pada pukul {random.randint(0, 23):02d}.{random.randint(0, 59):02d} WIB",
    "saat subuh tadi", "pada malam hari kemarin", "pada siang hari tadi", "seminggu yang lalu",
    "tiga hari yang lalu"
]
penutup_hoax = [
    "Menurut warga, ini adalah ulah alien yang ingin menguasai bumi.",
    "Namun, banyak yang tidak mempercayai kejadian ini.",
    "Berita ini ternyata hanya hoax yang menyebar di media sosial.",
    "Warga menjadi panik akibat berita ini, meskipun tidak ada bukti.",
    "Kejadian ini ternyata hanya prank dari sekelompok orang.",
    "Berita ini menjadi viral di media sosial, namun tidak ada konfirmasi resmi.",
    "Banyak yang mengatakan bahwa ini hanyalah cerita bohong belaka.",
    "Tidak ada bukti yang jelas mengenai kejadian ini hingga saat ini.",
    "Beberapa orang menyebut ini sebagai bagian dari konspirasi besar.",
    "Warga setempat mengaku hanya mendengar cerita ini dari grup WhatsApp."
]

# Fungsi untuk generate laporan valid
def generate_valid_report():
    bencana_pilihan = random.choice(bencana)
    if "Gempa" in bencana_pilihan:
        detail_bencana = f"{bencana_pilihan} dengan kekuatan {random.choice(magnitudo)}"
    else:
        detail_bencana = bencana_pilihan

    laporan = (f"{detail_bencana} melanda wilayah {random.choice(lokasi)} tepatnya di {random.choice(kecamatan)}, "
               f"{random.choice(desa)} {random.choice(waktu)}. {random.choice(dampak)}. {random.choice(penutup_valid)}")
    return laporan

# Fungsi untuk generate laporan invalid (hanya hoax, tanpa random)
def generate_invalid_report():
    laporan = (f"{random.choice(bencana_hoax)} terjadi {random.choice(lokasi_hoax)} {random.choice(waktu_hoax)}. "
               f"{random.choice(dampak_hoax)}. {random.choice(penutup_hoax)}")
    return laporan

# Generate dataset
n_samples = 10000  # 5000 valid + 5000 invalid
data = set()  # Pake set biar gak ada duplikat

# Generate laporan valid
while len(data) < n_samples // 2:
    laporan = generate_valid_report()
    data.add((laporan, "Valid"))

# Generate laporan invalid
while len(data) < n_samples:
    laporan = generate_invalid_report()
    data.add((laporan, "Invalid"))

# Convert set to list and shuffle
data = list(data)
random.shuffle(data)

# Masukkan ke Pandas DataFrame
df = pd.DataFrame(data, columns=["laporan", "label"])

# Tampilkan 10 baris pertama
print("10 Baris Pertama Dataset:")
print(df.head(10))

# Simpan ke CSV
df.to_csv("dataset_bencana_sintetis_clean_10000.csv", index=False)
print("\nDataset disimpan ke 'dataset_bencana_sintetis_clean_10000.csv'")

# Statistik sederhana
print("\nJumlah data per label:")
print(df['label'].value_counts())

# Statistik panjang laporan (untuk memastikan gak ada yang terlalu pendek)
df['panjang_laporan'] = df['laporan'].apply(lambda x: len(x.split()))
print("\nStatistik panjang laporan (jumlah kata):")
print(df['panjang_laporan'].describe())

10 Baris Pertama Dataset:
                                             laporan    label
0  Hujan emas yang membuat warga kaya mendadak te...  Invalid
1  Gempa bumi raksasa akibat ulah alien terjadi d...  Invalid
2  Banjir rob melanda wilayah Garut tepatnya di K...    Valid
3  Hujan emas yang membuat warga kaya mendadak te...  Invalid
4  Letusan gunung berapi melanda wilayah Lombok t...    Valid
5  Angin puting beliung melanda wilayah Lombok te...    Valid
6  Kekeringan akibat kutukan dukun terjadi di Kut...  Invalid
7  Gempa bumi dengan kekuatan 6.5 SR melanda wila...    Valid
8  Tsunami api yang membakar laut terjadi di Lang...  Invalid
9  Gempa bumi dengan kekuatan 5.0 SR melanda wila...    Valid

Dataset disimpan ke 'dataset_bencana_sintetis_clean_10000.csv'

Jumlah data per label:
label
Invalid    5000
Valid      5000
Name: count, dtype: int64

Statistik panjang laporan (jumlah kata):
count    10000.000000
mean        31.718400
std          2.979931
min         23.000000
25%       

In [1]:
import pandas as pd
import random

# Daftar yang sudah diperluas untuk meningkatkan variasi
bencana = [
    "Gempa bumi", "Banjir bandang", "Tanah longsor", "Tsunami", "Kekeringan", "Banjir",
    "Angin puting beliung", "Kebakaran hutan", "Letusan gunung berapi", "Banjir rob",
    "Longsor lahar dingin", "Bencana banjir", "Gelombang pasang", "Cuaca ekstrem",
    "Angin kencang", "Hujan deras terus-menerus", "Gempa susulan", "Banjir lumpur",
    "Erupsi gunung api", "Kebakaran lahan gambut"
]

magnitudo = [f"{random.uniform(3.0, 8.0):.1f} SR" for _ in range(100)]
lokasi = [
    "Yogyakarta", "Jakarta", "Bandung", "Garut", "Aceh", "Lombok", "Palu", "Bali",
    "Sumatra Barat", "Kalimantan Selatan", "Sulawesi Utara", "Papua", "Jawa Timur",
    "Banten", "Lampung", "NTT", "NTB", "Maluku", "Riau", "Bengkulu", "Jambi", "Sulawesi Selatan",
    "Kalimantan Barat", "Sumatra Utara", "Kepulauan Riau"
]

kecamatan = [f"Kecamatan {nama}" for nama in [
    "Cangkringan", "Cisarua", "Sukajaya", "Pamekasan", "Tanjung", "Ubud", "Soreang", "Ciputat",
    "Kalianda", "Woha", "Prambanan", "Karangasem", "Banyuwangi", "Serang", "Pesawaran",
    "Bojonggede", "Klojen", "Temanggung", "Wonosari", "Jatiasih"
]]

desa = [f"Desa {nama}" for nama in [
    "Sukamaju", "Mekarwangi", "Cikahuripan", "Pulosari", "Jaya Baru",
    "Tegalrejo", "Sidomulyo", "Gunungkidul", "Pasirjaya", "Margodadi",
    "Kertajaya", "Cibodas", "Sukawangi", "Sumberjaya", "Karangrejo"
]]

dampak = [
    f"Sebanyak {random.randint(10, 200)} rumah mengalami kerusakan ringan hingga berat",
    f"Tercatat {random.randint(1, 50)} warga mengalami luka-luka",
    f"Sedikitnya {random.randint(1, 20)} korban jiwa dilaporkan",
    "Akses jalan utama terputus akibat material longsor",
    "Pasokan listrik terhenti selama beberapa jam di wilayah terdampak",
    f"Lebih dari {random.randint(50, 500)} hektar lahan pertanian terendam banjir",
    f"Sebanyak {random.randint(1, 10)} desa terisolasi akibat akses jalan yang terputus",
    f"Sebanyak {random.randint(20, 300)} warga terpaksa mengungsi ke tempat yang lebih aman",
    "Banyak pohon tumbang yang menutup akses jalan utama",
    f"Kerugian material diperkirakan mencapai Rp {random.randint(1, 100)} miliar",
    f"Sebanyak {random.randint(1, 50)} ternak dilaporkan mati akibat bencana",
    "Pasokan air bersih terganggu di beberapa wilayah",
    f"Sebanyak {random.randint(1, 20)} sekolah mengalami kerusakan",
    "Jembatan utama yang menghubungkan dua desa ambruk",
    "Pelayanan kesehatan terganggu akibat kerusakan fasilitas",
    "Gangguan jaringan komunikasi menyulitkan proses koordinasi"
]

waktu = [
    "pada pagi hari ini", "pada malam hari tadi", "pada sore hari kemarin", "dua hari yang lalu",
    f"pada pukul {random.randint(0, 23):02d}.{random.randint(0, 59):02d} WIB",
    "saat subuh tadi", "pada malam hari kemarin", "pada siang hari tadi", "seminggu yang lalu",
    "tiga hari yang lalu"
]

penutup_valid = [
    "Pihak BPBD setempat langsung menangani kejadian ini dengan mendirikan posko darurat.",
    "Tim SAR dikerahkan ke lokasi untuk membantu proses evakuasi warga.",
    "Warga diminta tetap waspada terhadap potensi bencana susulan di wilayah tersebut.",
    "Pemerintah daerah segera menyalurkan bantuan logistik berupa makanan dan selimut.",
    "Hingga saat ini belum ada laporan korban jiwa, namun pendataan masih dilakukan.",
    "Proses evakuasi warga yang terdampak masih berlangsung hingga malam ini.",
    "Warga diimbau untuk tetap tenang dan mengikuti arahan dari petugas di lapangan.",
    "Bantuan darurat berupa tenda dan makanan mulai didistribusikan ke warga terdampak.",
    "Petugas masih melakukan pendataan kerugian dan korban di lokasi kejadian.",
    "Posko darurat telah didirikan untuk menampung warga yang kehilangan tempat tinggal."
]

# Elemen hoax tetap dipakai dari sebelumnya
bencana_hoax = [
    "Gempa bumi raksasa akibat ulah alien", "Banjir darah dari langit", "Tsunami api yang membakar laut",
    "Hujan meteor yang menghasilkan emas", "Angin topan membawa naga raksasa",
    "Kekeringan akibat kutukan dukun", "Lava meletus dari gunung es di tengah kota",
    "Hujan emas yang membuat warga kaya mendadak", "Banjir cokelat dari sungai",
    "Gempa bumi akibat ledakan di bulan", "Hujan berlian yang membuat warga takut",
    "Tsunami pelangi yang mengubah warna laut", "Angin topan ungu yang membuat orang terbang",
    "Kebakaran hutan akibat napas naga", "Letusan gunung berapi menghasilkan permen"
]
lokasi_hoax = [
    "di perairan Atlantis yang hilang", "di kota Bikini Bottom", "di Langit Ketujuh",
    "di dasar Laut Pasifik", "di permukaan Planet Mars", "di Kota Gaib Saranjana",
    "di Dunia Paralel", "di Kerajaan Atlantis yang tenggelam", "di Galaksi Andromeda",
    "di Benua Mu yang hilang", "di Kota Emas El Dorado", "di Lautan Awan",
    "di Dimensi Kelima", "di Kutub Selatan", "di pusat inti Bumi"
]
dampak_hoax = [
    "mengakibatkan semua ikan di laut mati mendadak", "menyebabkan alien muncul dari dalam air",
    "mengakibatkan dinosaurus hidup kembali di tengah kota", "menyebabkan semut berubah menjadi raksasa",
    "mengakibatkan pohon-pohon berubah menjadi emas", "menyebabkan langit berubah warna menjadi ungu",
    "mengakibatkan semua orang di wilayah itu bisa terbang", "menyebabkan ikan-ikan mulai berbicara",
    "mengakibatkan gunung berubah menjadi cokelat leleh", "menyebabkan warga panik dan ketakutan",
    "mengakibatkan burung-burung menjadi raksasa", "menyebabkan air laut berubah menjadi sirup manis",
    "mengakibatkan tanah berubah menjadi permen karet", "menyebabkan hewan-hewan mulai menari di jalan",
    "mengakibatkan semua warga menjadi takut dan bersembunyi"
]
penutup_hoax = [
    "Menurut warga, ini adalah ulah alien yang ingin menguasai bumi.",
    "Namun, banyak yang tidak mempercayai kejadian ini.",
    "Berita ini ternyata hanya hoax yang menyebar di media sosial.",
    "Warga menjadi panik akibat berita ini, meskipun tidak ada bukti.",
    "Kejadian ini ternyata hanya prank dari sekelompok orang.",
    "Berita ini menjadi viral di media sosial, namun tidak ada konfirmasi resmi.",
    "Banyak yang mengatakan bahwa ini hanyalah cerita bohong belaka.",
    "Tidak ada bukti yang jelas mengenai kejadian ini hingga saat ini.",
    "Beberapa orang menyebut ini sebagai bagian dari konspirasi besar.",
    "Warga setempat mengaku hanya mendengar cerita ini dari grup WhatsApp."
]

# Fungsi untuk generate laporan valid
def generate_valid_report():
    bencana_pilihan = random.choice(bencana)
    if "Gempa" in bencana_pilihan:
        detail_bencana = f"{bencana_pilihan} dengan kekuatan {random.choice(magnitudo)}"
    else:
        detail_bencana = bencana_pilihan

    laporan = (f"{detail_bencana} melanda wilayah {random.choice(lokasi)} tepatnya di {random.choice(kecamatan)}, "
               f"{random.choice(desa)} {random.choice(waktu)}. {random.choice(dampak)}. {random.choice(penutup_valid)}")
    return laporan

# Fungsi untuk generate laporan hoax
def generate_invalid_report():
    laporan = (f"{random.choice(bencana_hoax)} terjadi {random.choice(lokasi_hoax)} {random.choice(waktu)}. "
               f"{random.choice(dampak_hoax)}. {random.choice(penutup_hoax)}")
    return laporan

# Generate dataset 20000
n_samples = 20000
data = set()

# Generate valid reports
while len(data) < n_samples // 2:
    data.add((generate_valid_report(), "Valid"))

# Generate invalid reports
while len(data) < n_samples:
    data.add((generate_invalid_report(), "Invalid"))

# Convert and shuffle
data = list(data)
random.shuffle(data)

df = pd.DataFrame(data, columns=["laporan", "label"])

# Statistik panjang laporan
df['panjang_laporan'] = df['laporan'].apply(lambda x: len(x.split()))
df.head(), df['label'].value_counts(), df['panjang_laporan'].describe()



(                                             laporan    label  panjang_laporan
 0  Kebakaran lahan gambut melanda wilayah Sumatra...    Valid               35
 1  Kebakaran lahan gambut melanda wilayah Kaliman...    Valid               38
 2  Gempa susulan dengan kekuatan 5.9 SR melanda w...    Valid               37
 3  Tsunami api yang membakar laut terjadi di Kutu...  Invalid               31
 4  Hujan berlian yang membuat warga takut terjadi...  Invalid               26,
 label
 Valid      10000
 Invalid    10000
 Name: count, dtype: int64,
 count    20000.000000
 mean        31.686550
 std          2.974919
 min         22.000000
 25%         30.000000
 50%         32.000000
 75%         34.000000
 max         42.000000
 Name: panjang_laporan, dtype: float64)

In [32]:
import pandas as pd
import random

# Elemen untuk laporan valid
bencana = [
    "Gempa bumi", "Banjir bandang", "Tanah longsor", "Tsunami", "Kekeringan", "Banjir",
    "Angin puting beliung", "Kebakaran hutan", "Letusan gunung berapi", "Banjir rob",
    "Longsor lahar dingin", "Bencana banjir"
]
magnitudo = [f"{random.uniform(3.0, 8.0):.1f} SR" for _ in range(100)]
lokasi = [
    "Yogyakarta", "Jakarta", "Bandung", "Garut", "Aceh", "Lombok", "Palu", "Bali",
    "Sumatra Barat", "Kalimantan Selatan", "Sulawesi Utara", "Papua", "Jawa Timur",
    "Banten", "Lampung", "NTT", "NTB", "Maluku", "Riau", "Bengkulu"
]
kecamatan = [
    "Kecamatan Cangkringan", "Kecamatan Cisarua", "Kecamatan Sukajaya", "Kecamatan Pamekasan",
    "Kecamatan Tanjung", "Kecamatan Ubud", "Kecamatan Soreang", "Kecamatan Ciputat",
    "Kecamatan Kalianda", "Kecamatan Woha", "Kecamatan Prambanan", "Kecamatan Karangasem",
    "Kecamatan Banyuwangi", "Kecamatan Serang", "Kecamatan Pesawaran"
]
desa = [
    "Desa Sukamaju", "Desa Mekarwangi", "Desa Cikahuripan", "Desa Pulosari", "Desa Jaya Baru",
    "Desa Tegalrejo", "Desa Sidomulyo", "Desa Gunungkidul", "Desa Pasirjaya", "Desa Margodadi"
]
dampak = [
    f"Sebanyak {random.randint(10, 200)} rumah mengalami kerusakan ringan hingga berat",
    f"Tercatat {random.randint(1, 50)} warga mengalami luka-luka",
    f"Sedikitnya {random.randint(1, 20)} korban jiwa dilaporkan",
    "Akses jalan utama terputus akibat material longsor",
    "Pasokan listrik terhenti selama beberapa jam di wilayah terdampak",
    f"Lebih dari {random.randint(50, 500)} hektar lahan pertanian terendam banjir",
    f"Sebanyak {random.randint(1, 10)} desa terisolasi akibat akses jalan yang terputus",
    f"Sebanyak {random.randint(20, 300)} warga terpaksa mengungsi ke tempat yang lebih aman",
    "Banyak pohon tumbang yang menutup akses jalan utama",
    f"Kerugian material diperkirakan mencapai Rp {random.randint(1, 100)} miliar",
    f"Sebanyak {random.randint(1, 50)} ternak dilaporkan mati akibat bencana",
    "Pasokan air bersih terganggu di beberapa wilayah",
    f"Sebanyak {random.randint(1, 20)} sekolah mengalami kerusakan",
    "Jembatan utama yang menghubungkan dua desa ambruk"
]
waktu = [
    "pada pagi hari ini", "pada malam hari tadi", "pada sore hari kemarin", "dua hari yang lalu",
    f"pada pukul {random.randint(0, 23):02d}.{random.randint(0, 59):02d} WIB",
    "saat subuh tadi", "pada malam hari kemarin", "pada siang hari tadi", "seminggu yang lalu",
    "tiga hari yang lalu"
]
penutup_valid = [
    "Pihak BPBD setempat langsung menangani kejadian ini dengan mendirikan posko darurat.",
    "Tim SAR dikerahkan ke lokasi untuk membantu proses evakuasi warga.",
    "Warga diminta tetap waspada terhadap potensi bencana susulan di wilayah tersebut.",
    "Pemerintah daerah segera menyalurkan bantuan logistik berupa makanan dan selimut.",
    "Hingga saat ini belum ada laporan korban jiwa, namun pendataan masih dilakukan.",
    "Proses evakuasi warga yang terdampak masih berlangsung hingga malam ini.",
    "Warga diimbau untuk tetap tenang dan mengikuti arahan dari petugas di lapangan.",
    "Bantuan darurat berupa tenda dan makanan mulai didistribusikan ke warga terdampak.",
    "Petugas masih melakukan pendataan kerugian dan korban di lokasi kejadian.",
    "Posko darurat telah didirikan untuk menampung warga yang kehilangan tempat tinggal."
]

# Template tambahan laporan valid
template_valid = [
    "Telah terjadi {bencana} pada daerah {lokasi}.",
    "Telah terjadi {bencana} pada daerah {lokasi}, tepatnya di {kecamatan}, {desa} {waktu}.",
    "Gempa bumi dahsyat di kabupaten {lokasi}.",
    "Banjir bandang dahsyat di kabupaten {lokasi} pada hari Sabtu kemarin.",
    "Telah terjadi kerusakan besar akibat gunung meletus pada 17 Maret 2025 di {lokasi}.",
    "Lapor pak, telah terjadi banjir di daerah {lokasi}.",
    "Banjir telah terjadi akibat kali tidak mampu menampungnya di wilayah {lokasi}.",
    "Tanah longsor akibat bencana alam yang terjadi di {kecamatan}, {desa}, {lokasi}.",
    "Hujan deras mengakibatkan banyak rumah warga terkena dampak banjirnya di {lokasi}."
]

# Laporan invalid (hoax)
bencana_hoax = [
    "Gempa bumi raksasa akibat ulah alien", "Banjir darah dari langit", "Tsunami api yang membakar laut",
    "Hujan meteor yang menghasilkan emas", "Angin topan membawa naga raksasa",
    "Kekeringan akibat kutukan dukun", "Lava meletus dari gunung es di tengah kota",
    "Hujan emas yang membuat warga kaya mendadak", "Banjir cokelat dari sungai",
    "Gempa bumi akibat ledakan di bulan", "Hujan berlian yang membuat warga takut",
    "Tsunami pelangi yang mengubah warna laut", "Angin topan ungu yang membuat orang terbang",
    "Kebakaran hutan akibat napas naga", "Letusan gunung berapi menghasilkan permen"
]
lokasi_hoax = [
    "di perairan Atlantis yang hilang", "di kota Bikini Bottom", "di Langit Ketujuh",
    "di dasar Laut Pasifik", "di permukaan Planet Mars", "di Kota Gaib Saranjana",
    "di Dunia Paralel", "di Kerajaan Atlantis yang tenggelam", "di Galaksi Andromeda",
    "di Benua Mu yang hilang", "di Kota Emas El Dorado", "di Lautan Awan",
    "di Dimensi Kelima", "di Kutub Selatan", "di pusat inti Bumi"
]
dampak_hoax = [
    "mengakibatkan semua ikan di laut mati mendadak", "menyebabkan alien muncul dari dalam air",
    "mengakibatkan dinosaurus hidup kembali di tengah kota", "menyebabkan semut berubah menjadi raksasa",
    "mengakibatkan pohon-pohon berubah menjadi emas", "menyebabkan langit berubah warna menjadi ungu",
    "mengakibatkan semua orang di wilayah itu bisa terbang", "menyebabkan ikan-ikan mulai berbicara",
    "mengakibatkan gunung berubah menjadi cokelat leleh", "menyebabkan warga panik dan ketakutan",
    "mengakibatkan burung-burung menjadi raksasa", "menyebabkan air laut berubah menjadi sirup manis",
    "mengakibatkan tanah berubah menjadi permen karet", "menyebabkan hewan-hewan mulai menari di jalan",
    "mengakibatkan semua warga menjadi takut dan bersembunyi"
]
penutup_hoax = [
    "Menurut warga, ini adalah ulah alien yang ingin menguasai bumi.",
    "Namun, banyak yang tidak mempercayai kejadian ini.",
    "Berita ini ternyata hanya hoax yang menyebar di media sosial.",
    "Warga menjadi panik akibat berita ini, meskipun tidak ada bukti.",
    "Kejadian ini ternyata hanya prank dari sekelompok orang.",
    "Berita ini menjadi viral di media sosial, namun tidak ada konfirmasi resmi.",
    "Banyak yang mengatakan bahwa ini hanyalah cerita bohong belaka.",
    "Tidak ada bukti yang jelas mengenai kejadian ini hingga saat ini.",
    "Beberapa orang menyebut ini sebagai bagian dari konspirasi besar.",
    "Warga setempat mengaku hanya mendengar cerita ini dari grup WhatsApp."
]

# Fungsi laporan valid
def generate_valid_report():
    if random.random() < 0.5:
        template = random.choice(template_valid)
        laporan = template.format(
            bencana=random.choice(bencana),
            lokasi=random.choice(lokasi),
            kecamatan=random.choice(kecamatan),
            desa=random.choice(desa),
            waktu=random.choice(waktu)
        )
        laporan += " " + random.choice(penutup_valid)
    else:
        bencana_pilihan = random.choice(bencana)
        if "Gempa" in bencana_pilihan:
            detail_bencana = f"{bencana_pilihan} dengan kekuatan {random.choice(magnitudo)}"
        else:
            detail_bencana = bencana_pilihan
        laporan = (f"{detail_bencana} melanda wilayah {random.choice(lokasi)} tepatnya di {random.choice(kecamatan)}, "
                   f"{random.choice(desa)} {random.choice(waktu)}. {random.choice(dampak)}. {random.choice(penutup_valid)}")
    return laporan

# Fungsi laporan hoax
def generate_invalid_report():
    return (f"{random.choice(bencana_hoax)} terjadi {random.choice(lokasi_hoax)} {random.choice(waktu)}. "
            f"{random.choice(dampak_hoax)}. {random.choice(penutup_hoax)}")

# Generate dataset
n_samples = 20000
data = set()

while len(data) < n_samples // 2:
    laporan = generate_valid_report()
    data.add((laporan, "Valid"))

while len(data) < n_samples:
    laporan = generate_invalid_report()
    data.add((laporan, "Invalid"))

data = list(data)
random.shuffle(data)
df = pd.DataFrame(data, columns=["laporan", "label"])

# Simpan & statistik
df.to_csv("dataset_bencana_sintetis_clean_20000.csv", index=False)
print("\nDataset disimpan ke 'dataset_bencana_sintetis_clean_20000.csv'")
print("\nJumlah data per label:")
print(df['label'].value_counts())
df['panjang_laporan'] = df['laporan'].apply(lambda x: len(x.split()))
print("\nStatistik panjang laporan (jumlah kata):")
print(df['panjang_laporan'].describe())



Dataset disimpan ke 'dataset_bencana_sintetis_clean_20000.csv'

Jumlah data per label:
label
Valid      10000
Invalid    10000
Name: count, dtype: int64

Statistik panjang laporan (jumlah kata):
count    20000.000000
mean        29.801100
std          4.473269
min         16.000000
25%         28.000000
50%         30.000000
75%         33.000000
max         43.000000
Name: panjang_laporan, dtype: float64


In [2]:
df.head()

,laporan,label,panjang_laporan
0,Kebakaran lahan gambut melanda wilayah Sumatra...,Valid,35
1,Kebakaran lahan gambut melanda wilayah Kaliman...,Valid,38
2,Gempa susulan dengan kekuatan 5.9 SR melanda w...,Valid,37
3,Tsunami api yang membakar laut terjadi di Kutu...,Invalid,31
4,Hujan berlian yang membuat warga takut terjadi...,Invalid,26


In [3]:
df.to_csv("v1_txt.csv")

In [4]:
!pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.8 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.metrics import accuracy_score

pd.options.mode.chained_assignment = None
np.random.seed(0)

In [33]:
df = pd.read_csv("dataset_bencana_sintetis_clean_20000.csv")

In [ ]:
# df = pd.read_csv('https://raw.githubusercontent.com/cryoras/natural_disaster_validation/refs/heads/main/dataset.csv')
# df =df.sample(n=10000)

In [34]:
df.head()

,laporan,label
0,Telah terjadi kerusakan besar akibat gunung me...,Valid
1,Tsunami api yang membakar laut terjadi di pusa...,Invalid
2,Tanah longsor akibat bencana alam yang terjadi...,Valid
3,Gempa bumi dahsyat di kabupaten NTB. Tim SAR d...,Valid
4,Angin puting beliung melanda wilayah Bandung t...,Valid


In [35]:
df['label'].value_counts()

,count
label,
Valid,10000
Invalid,10000


#preprocessing data

In [36]:
df = df.drop_duplicates()
df.shape


(20000, 2)

In [37]:
df['label'].value_counts()

,count
label,
Valid,10000
Invalid,10000


In [38]:
def cleaningText(t):
  t = re.sub(r'[0-9]+', '', t)
  t = re.sub(r'[^\w\s]', '', t)
  t = t.replace('\n', ' ') # mengganti baris baru dengan spasi
  t = t.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
  t = t.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
  return t

def caseFold(t):
  t = t.lower()
  return t

def tokenizeText(t):
  t = word_tokenize(t)
  return t

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(t):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  words = t.split()
  stemmed_words = [stemmer.stem(word) for word in words]
  stemmed_text = ' '.join(stemmed_words)
  return stemmed_text

def toSentence(list_words):
  sentence = ' '.join(word for word in list_words)
  return sentence

lang_dict = {
    # Istilah darurat tidak baku
    "gempa": "gempabumi",
    "gempa tektonik": "gempabumi",
    "gempa bumi": "gempabumi",
    "banjir bandang": "banjir",
    "longsor": "tanah longsor",
    "tsu": "tsunami",
    "angin puting": "angin puting beliung",
    "kebakaran hutan": "karhutla",

    # Singkatan darurat
    "bpbd": "Badan Penanggulangan Bencana Daerah",
    "posko": "posko bencana",
    "pengungsi": "korban terdampak",
    "mksd": "maksud",
    "jln": "jalan",

    # Ekspresi populer
    "wih": "waduh",
    "parah": "berat",
    "kacau": "rusak parah",
    "gede": "besar",
    "ampe": "sampai",
    "bnyk": "banyak",
    "krn": "karena",
    "yg": "yang"
}

def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in lang_dict:
            fixed_words.append(lang_dict[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [39]:
nltk.download('punkt_tab')
nltk.download('stopwords')
df['text_clean'] = df['laporan'].apply(cleaningText)
df['text_casefold'] = df['text_clean'].apply(caseFold)
df['text_slang'] = df['text_casefold'].apply(fix_slangwords)
df['text_tokenize'] = df['text_slang'].apply(tokenizeText)
df['text_stopword'] = df['text_tokenize'].apply(filteringText)
df['text_end'] = df['text_stopword'].apply(toSentence)



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
df.head()

,laporan,label,text_clean,text_casefold,text_slang,text_tokenize,text_stopword,text_end
0,Telah terjadi kerusakan besar akibat gunung me...,Valid,Telah terjadi kerusakan besar akibat gunung me...,telah terjadi kerusakan besar akibat gunung me...,telah terjadi kerusakan besar akibat gunung me...,"[telah, terjadi, kerusakan, besar, akibat, gun...","[kerusakan, akibat, gunung, meletus, maret, ri...",kerusakan akibat gunung meletus maret riau war...
1,Tsunami api yang membakar laut terjadi di pusa...,Invalid,Tsunami api yang membakar laut terjadi di pusa...,tsunami api yang membakar laut terjadi di pusa...,tsunami api yang membakar laut terjadi di pusa...,"[tsunami, api, yang, membakar, laut, terjadi, ...","[tsunami, api, membakar, laut, pusat, inti, bu...",tsunami api membakar laut pusat inti bumi mala...
2,Tanah longsor akibat bencana alam yang terjadi...,Valid,Tanah longsor akibat bencana alam yang terjadi...,tanah longsor akibat bencana alam yang terjadi...,tanah tanah longsor akibat bencana alam yang t...,"[tanah, tanah, longsor, akibat, bencana, alam,...","[tanah, tanah, longsor, akibat, bencana, alam,...",tanah tanah longsor akibat bencana alam kecama...
3,Gempa bumi dahsyat di kabupaten NTB. Tim SAR d...,Valid,Gempa bumi dahsyat di kabupaten NTB Tim SAR di...,gempa bumi dahsyat di kabupaten ntb tim sar di...,gempabumi bumi dahsyat di kabupaten ntb tim sa...,"[gempabumi, bumi, dahsyat, di, kabupaten, ntb,...","[gempabumi, bumi, dahsyat, kabupaten, ntb, tim...",gempabumi bumi dahsyat kabupaten ntb tim sar d...
4,Angin puting beliung melanda wilayah Bandung t...,Valid,Angin puting beliung melanda wilayah Bandung t...,angin puting beliung melanda wilayah bandung t...,angin puting beliung melanda wilayah bandung t...,"[angin, puting, beliung, melanda, wilayah, ban...","[angin, puting, beliung, melanda, wilayah, ban...",angin puting beliung melanda wilayah bandung t...


In [41]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['label_encoded'] = encoder.fit_transform(df['label'])

In [42]:
df['label'].value_counts()

,count
label,
Valid,10000
Invalid,10000


In [43]:
df['label_encoded'].value_counts()

,count
label_encoded,
1,10000
0,10000


In [44]:
df.head()

,laporan,label,text_clean,text_casefold,text_slang,text_tokenize,text_stopword,text_end,label_encoded
0,Telah terjadi kerusakan besar akibat gunung me...,Valid,Telah terjadi kerusakan besar akibat gunung me...,telah terjadi kerusakan besar akibat gunung me...,telah terjadi kerusakan besar akibat gunung me...,"[telah, terjadi, kerusakan, besar, akibat, gun...","[kerusakan, akibat, gunung, meletus, maret, ri...",kerusakan akibat gunung meletus maret riau war...,1
1,Tsunami api yang membakar laut terjadi di pusa...,Invalid,Tsunami api yang membakar laut terjadi di pusa...,tsunami api yang membakar laut terjadi di pusa...,tsunami api yang membakar laut terjadi di pusa...,"[tsunami, api, yang, membakar, laut, terjadi, ...","[tsunami, api, membakar, laut, pusat, inti, bu...",tsunami api membakar laut pusat inti bumi mala...,0
2,Tanah longsor akibat bencana alam yang terjadi...,Valid,Tanah longsor akibat bencana alam yang terjadi...,tanah longsor akibat bencana alam yang terjadi...,tanah tanah longsor akibat bencana alam yang t...,"[tanah, tanah, longsor, akibat, bencana, alam,...","[tanah, tanah, longsor, akibat, bencana, alam,...",tanah tanah longsor akibat bencana alam kecama...,1
3,Gempa bumi dahsyat di kabupaten NTB. Tim SAR d...,Valid,Gempa bumi dahsyat di kabupaten NTB Tim SAR di...,gempa bumi dahsyat di kabupaten ntb tim sar di...,gempabumi bumi dahsyat di kabupaten ntb tim sa...,"[gempabumi, bumi, dahsyat, di, kabupaten, ntb,...","[gempabumi, bumi, dahsyat, kabupaten, ntb, tim...",gempabumi bumi dahsyat kabupaten ntb tim sar d...,1
4,Angin puting beliung melanda wilayah Bandung t...,Valid,Angin puting beliung melanda wilayah Bandung t...,angin puting beliung melanda wilayah bandung t...,angin puting beliung melanda wilayah bandung t...,"[angin, puting, beliung, melanda, wilayah, ban...","[angin, puting, beliung, melanda, wilayah, ban...",angin puting beliung melanda wilayah bandung t...,1


# model building

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, precision_score

In [46]:
X = df['text_end']
Y = df['label_encoded']

In [47]:
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

In [48]:
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())
features_df

,air,akibat,akses,alam,alien,andromeda,angin,api,arahan,atlantis,...,ulah,ungu,utama,viral,warga,warna,waspada,whatsapp,wib,wilayah
0,0.000000,0.212789,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.152807,0.0,0.387148,0.000000,0.0,0.189128
1,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.314452,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
2,0.000000,0.189024,0.0,0.363712,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.135741,0.0,0.000000,0.000000,0.0,0.000000
3,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.145917,0.0,0.000000,0.000000,0.0,0.000000
4,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.227511,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.109577,0.0,0.000000,0.000000,0.0,0.135622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.242533,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.113009,0.0,0.286316,0.000000,0.0,0.419611
19996,0.216987,0.140793,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.255966,0.000000,0.0,0.000000,0.000000,0.0,0.000000
19997,0.000000,0.151278,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.108635,0.0,0.000000,0.270648,0.0,0.000000
19998,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.180655


In [49]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, Y, test_size=0.2, random_state=42)

In [50]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [51]:
from sklearn.metrics import classification_report, accuracy_score

In [52]:
naive_bayes = BernoulliNB()
naive_bayes.fit(X_train.toarray(), y_train)
y_pred_train_nb = naive_bayes.predict(X_train.toarray())
y_pred_test_nb = naive_bayes.predict(X_test.toarray())
accuracy_train_nb = accuracy_score(y_pred_train_nb, y_train)
accuracy_test_nb = accuracy_score(y_pred_test_nb, y_test)
print('Naive Bayes - accuracy_train:', accuracy_train_nb)
print('Naive Bayes - accuracy_test:', accuracy_test_nb)
print("\nClassification Report:")
print(classification_report(y_test, y_pred_test_nb, target_names=["Invalid", "Valid"]))

Naive Bayes - accuracy_train: 1.0
Naive Bayes - accuracy_test: 1.0

Classification Report:
              precision    recall  f1-score   support

     Invalid       1.00      1.00      1.00      2034
       Valid       1.00      1.00      1.00      1966

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



In [53]:
random_forest = RandomForestClassifier()
random_forest.fit(X_train.toarray(), y_train)
y_pred_train_rf = random_forest.predict(X_train.toarray())
y_pred_test_rf = random_forest.predict(X_test.toarray())
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)
print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)
print(classification_report(y_test, y_pred_test_rf, target_names=["Invalid", "Valid"]))

Random Forest - accuracy_train: 1.0
Random Forest - accuracy_test: 1.0
              precision    recall  f1-score   support

     Invalid       1.00      1.00      1.00      2034
       Valid       1.00      1.00      1.00      1966

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



In [54]:
logistic_regression = LogisticRegression()
logistic_regression.fit(X_train.toarray(), y_train)
y_pred_train_lr = logistic_regression.predict(X_train.toarray())
y_pred_test_lr = logistic_regression.predict(X_test.toarray())
accuracy_train_lr = accuracy_score(y_pred_train_lr, y_train)
accuracy_test_lr = accuracy_score(y_pred_test_lr, y_test)
print('Logistic Regression - accuracy_train:', accuracy_train_lr)
print('Logistic Regression - accuracy_test:', accuracy_test_lr)
print(classification_report(y_test, y_pred_test_lr, target_names=["Invalid", "Valid"]))

Logistic Regression - accuracy_train: 1.0
Logistic Regression - accuracy_test: 1.0
              precision    recall  f1-score   support

     Invalid       1.00      1.00      1.00      2034
       Valid       1.00      1.00      1.00      1966

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



In [55]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train.toarray(), y_train)
y_pred_train_dt = decision_tree.predict(X_train.toarray())
y_pred_test_dt = decision_tree.predict(X_test.toarray())
accuracy_train_dt = accuracy_score(y_pred_train_dt, y_train)
accuracy_test_dt = accuracy_score(y_pred_test_dt, y_test)
print('Decision Tree - accuracy_train:', accuracy_train_dt)
print('Decision Tree - accuracy_test:', accuracy_test_dt)
print(classification_report(y_test, y_pred_test_dt, target_names=["Invalid", "Valid"]))

Decision Tree - accuracy_train: 1.0
Decision Tree - accuracy_test: 0.999
              precision    recall  f1-score   support

     Invalid       1.00      1.00      1.00      2034
       Valid       1.00      1.00      1.00      1966

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



# inference

In [56]:
kalimat_baru = input("Masukkan kalimat baru: ")

kalimat_baru_cleaned = cleaningText(kalimat_baru)
kalimat_baru_casefolded = caseFold(kalimat_baru_cleaned)
kalimat_baru_slangfixed = fix_slangwords(kalimat_baru_casefolded)
kalimat_baru_tokenized = tokenizeText(kalimat_baru_slangfixed)
kalimat_baru_filtered = filteringText(kalimat_baru_tokenized)
kalimat_baru_final = toSentence(kalimat_baru_filtered)

X_kalimat_baru = tfidf.transform([kalimat_baru_final])

# Memperoleh prediksi sentimen kalimat baru
prediksi_sentimen = logistic_regression.predict(X_kalimat_baru)

print(prediksi_sentimen)

Masukkan kalimat baru: telah terjadi bencana banjir di kabupaten serbabuk
[1]


In [57]:
kalimat_baru = input("Masukkan kalimat baru: ")

kalimat_baru_cleaned = cleaningText(kalimat_baru)
kalimat_baru_casefolded = caseFold(kalimat_baru_cleaned)
kalimat_baru_slangfixed = fix_slangwords(kalimat_baru_casefolded)
kalimat_baru_tokenized = tokenizeText(kalimat_baru_slangfixed)
kalimat_baru_filtered = filteringText(kalimat_baru_tokenized)
kalimat_baru_final = toSentence(kalimat_baru_filtered)

X_kalimat_baru = tfidf.transform([kalimat_baru_final])
prediksi_sentimen = logistic_regression.predict(X_kalimat_baru)

print(prediksi_sentimen)

Masukkan kalimat baru:  telah terjadi banjir di kabupaten serbabuk
[1]


# save model

In [58]:
import joblib
# x variabel model
# y nama model
joblib.dump(logistic_regression,"v1_text.joblib")

['v1_text.joblib']